In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dont-overfit-ii/sample_submission.csv
/kaggle/input/dont-overfit-ii/train.csv
/kaggle/input/dont-overfit-ii/test.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [4]:
df = pd.read_csv('/kaggle/input/dont-overfit-ii/train.csv')
df = df.drop('id',axis=1)
y = df['target']
X = df.drop('target',axis=1)
test = pd.read_csv('/kaggle/input/dont-overfit-ii/test.csv')
test.drop('id',axis=1,inplace=True)
X_mean = X.mean(axis=1)
X['mean']  = X_mean
test_mean = test.mean(axis=1)
test['mean'] = test_mean

In [5]:
features = [ '16', '33', '43', '45', '52', '63', '65', '73', '90', '91', '117', '133', '134', '149', '189', '199', '217', '237', '258', '295']
X1 = X[features]
test = test[features]
print(len(X1), len(test))

250 19750


In [6]:
params = {}
params['random_state'] = 0
params['n_jobs'] = -1
params['C'] = 0.126
params['penalty'] = 'l1'
params['class_weight'] = 'balance'
params['solver'] = 'saga'
best = params

In [7]:
def train(X1, y, best):
    folds = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 20, random_state = 0)
    oof =  np.zeros(len(X1))
    clfs = []
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X1, y)):
            #print('--- Fold {} started at {}'.format(n_fold, time.ctime()))

            train_x, train_y = X1.iloc[train_idx], y.iloc[train_idx]
            valid_x, valid_y = X1.iloc[valid_idx], y.iloc[valid_idx]

            clf = LogisticRegression(**best)
            clf.fit(train_x, train_y)
            clfs.append(clf)
            
            oof[valid_idx] += clf.predict_proba(valid_x)[:,1]
            
    return clfs, oof

In [8]:
def predict(clfs,test):
    predictions = np.zeros(len(test))
    for c in clfs:
        predictions += c.predict_proba(test)[:,1]
    predictions = predictions/len(clfs)
    return predictions

In [9]:
clfs, oof = train(X1 ,y, best)
roc_auc_score(y, oof/20)

0.5859228447924313

In [10]:
predicitons =  predict(clfs,test)

In [12]:
result = pd.read_csv('/kaggle/input/dont-overfit-ii/sample_submission.csv')
result['target'] = predicitons
result.to_csv('submission13.csv', index=False)

In [29]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam
from keras.utils import plot_model
#from keras import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [28]:
#pip install keras
import keras

In [30]:
es_callback = keras.callbacks.EarlyStopping(monitor='auc', patience=80)

In [31]:
model = keras.Sequential()
model.add(keras.layers.Dense(16, activation='relu', input_dim=len(features)))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X1,y,batch_size=40, 
                    epochs=290, validation_split=0.25, shuffle=True, callbacks=[es_callback])

Epoch 1/290
5/5 [==============================] - 1s 62ms/step - loss: 0.8167 - accuracy: 0.5775 - val_loss: 0.7550 - val_accuracy: 0.5397
Epoch 2/290
5/5 [==============================] - 0s 9ms/step - loss: 0.8074 - accuracy: 0.5615 - val_loss: 0.7490 - val_accuracy: 0.5556
Epoch 3/290
5/5 [==============================] - 0s 10ms/step - loss: 0.8102 - accuracy: 0.5775 - val_loss: 0.7443 - val_accuracy: 0.5714
Epoch 4/290
5/5 [==============================] - 0s 9ms/step - loss: 0.7567 - accuracy: 0.5936 - val_loss: 0.7404 - val_accuracy: 0.5714
Epoch 5/290
5/5 [==============================] - 0s 10ms/step - loss: 0.6948 - accuracy: 0.6150 - val_loss: 0.7375 - val_accuracy: 0.5873
Epoch 6/290
5/5 [==============================] - 0s 10ms/step - loss: 0.7130 - accuracy: 0.6150 - val_loss: 0.7347 - val_accuracy: 0.6032
Epoch 7/290
5/5 [==============================] - 0s 9ms/step - loss: 0.7819 - accuracy: 0.5775 - val_loss: 0.7325 - val_accuracy: 0.6032
Epoch 8/290
5/5 [======

In [32]:
predictions = model.predict(test)

618/618 [==============================] - 1s 1ms/step


In [33]:
predictions

array([[0.14265068],
       [0.62688476],
       [0.509202  ],
       ...,
       [0.13116474],
       [0.54472274],
       [0.26901475]], dtype=float32)

In [34]:
result = pd.read_csv('/kaggle/input/dont-overfit-ii/sample_submission.csv')
result['target'] = predictions
result.to_csv('submission11.csv', index = False)